#### Détourage des cellules

> Choix d'enregistrer les images des cellules détourées pour modèle de prédiction de type 
>
> Test réalisé de détourage via prédiction par le modèle de segmentation au moment de l'entrainement d'un modèle de prédiction : temps d'entrainement trop long

In [8]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import numpy as np
import pandas as pd
import cv2

In [9]:
data = pd.read_csv('../base_10_class.csv',index_col=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31735 entries, 0 to 15516
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_path  31735 non-null  object
 1   filename   31735 non-null  object
 2   class      31735 non-null  object
dtypes: object(3)
memory usage: 991.7+ KB


In [10]:
from keras.models import load_model
model_mask = load_model('segmentation_fcn')

In [11]:
# Process pour detourage
# Fonction preprocess pour les images à detourer (modèle de masque)
def load_and_preprocess_mask(img_filepath):
    img = tf.io.read_file(img_filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, [128, 128]) 
    img = tf.cast(img, tf.float32) / 255.0
    return img

# Création du masque à partir de la prédiction, mise au format nécessaire
def create_mask(pred_mask,resize):
    mask = tf.argmax(pred_mask, axis=-1) 
    mask = mask[..., tf.newaxis]
    mask = tf.image.resize(mask,resize)
    return tf.squeeze(mask[0],-1)
# Detourage de la cellule en utilisant la prédiction du modèle de masque

def cut_contour(img_filepath,model,resize):
    img_to_mask = load_and_preprocess_mask(img_filepath)
    pred_mask = model(img_to_mask[tf.newaxis, ...])
    mask = create_mask(pred_mask,resize)
    img = tf.io.read_file(img_filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, resize) # => Float en même temps
    # Supprime autour de la cellule => noir
    mask_bg = (mask == 0)[..., tf.newaxis]
    mask_bg  = tf.cast(mask_bg, tf.float32)
    mask_bg  = tf.concat([mask_bg, mask_bg, mask_bg], 2)
    return (img *  mask_bg )


In [12]:
def load_and_preprocess(filename, model, resize=(255,255)):
    # Detourage
    img = cut_contour(filename,model,resize)
    return img

In [13]:
AUTO = tf.data.experimental.AUTOTUNE

In [14]:
dataset = tf.data.Dataset.from_tensor_slices((data['file_path'],data['filename']))
dataset = (dataset.map(lambda x, y: [load_and_preprocess(x, model_mask),y], num_parallel_calls=AUTO))

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [15]:
#!mkdir ..\images_detour # Windows
!mkdir ../images_detour

In [16]:
# Enregistrement des images détourées sans sous dossier
for image, filename in dataset.take(-1):
     cv2.imwrite('../images_detour/'+tf.compat.as_str_any(filename.numpy()), tf.cast(image, tf.int16).numpy())

### Vérification du détourage

> Suppression de toutes les globules rouges et du fond, mais quelques fois rognage de la cellule
>
> 132 images supprimées suites à vérification car anomalies

In [18]:
# Récupère le nom de toutes les images sauvegardées => suppression dans la bd
import os
files=[]
for file in os.scandir('../images_detour'):
    files.append(file.name)

In [19]:
len(files)

31603

In [21]:
data[data['filename'].isin(files)].reset_index().to_csv('../files_detour.csv')